# SDND Project 1 - Finding Lane Lines

In [6]:
# 0. Import
import sys
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 1. Canny Edge Detection
def Canny_Edge_Detection(frame):
    
    # 1.1. Convert original image frame to grayscale image
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    # 1.2. Gaussian blur the grayscale image
    blur_gray = cv2.GaussianBlur(gray_frame, (5, 5), 0)
    # 1.3. Perform Canny over blurred image
    canny_blur = cv2.Canny(blur_gray, 50, 160)

    return canny_blur

# 2. Region of Interest (ROI)
def Region_of_Interest(frame):
    
    # 2.1. Get height and width
    height = frame.shape[0]
    width = frame.shape[1]
    # 2.2. Set ROI 
    # polygon = np.array([[(int(width*(3/20)), height), (int(width*(17/20)), height), (int(width/2), int(height*(2/5)))]])
    polygon = np.array([[(60, height), (900, height), (480, 280)]])
    # 2.3. Mask original frame with polygon
    mask = np.zeros_like(frame)
    cv2.fillPoly(mask, polygon, 255)
    masked_frame = cv2.bitwise_and(frame, mask)
    
    return masked_frame

# 3. Visualize Detected Lane Lines
# 3.1. Draw the lines
def Display_Lines(frame, lines):
    
    # Draw detected lane lines on the frame
    lined_frame = np.zeros_like(frame)

    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line.reshape(4)
            try:
                cv2.line(lined_frame, (x1, y1), (x2, y2), (0, 255, 0), 5)    
                continue
            except:
                # print(x1, x2, y1, y2)
                continue

    return lined_frame

# 3.2. Make coordinates for lines
def Cartesian_Coordinates(frame, line_params):
    
    slope, intercept = line_params
    
    y1 = frame.shape[0]
    y2 = int(y1 * (13/20))
    
    x1 = int((y1 - intercept) / slope)
    x2 = int((y2 - intercept) / slope)    
    
    return np.array([x1, y1, x2, y2])

# 3.3. Find average lines' for drawing
def Average_Params(frame, lines):
    
    left_fit = []
    right_fit = []

    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line.reshape(4)

            try:
                params = np.polyfit((x1, x2), (y1, y2), 1)  
            
                slope = params[0]
                intercept = params[1]
                
                if slope < 0:
                    left_fit.append((slope, intercept))
                else:
                    right_fit.append((slope, intercept))
                    
                if len(left_fit) and len(right_fit):
                    
                    left_fit_average = np.average(left_fit, axis=0)
                    right_fit_average = np.average(right_fit, axis=0)

                    left_line = Cartesian_Coordinates(frame, left_fit_average)
                    right_line = Cartesian_Coordinates(frame, right_fit_average)

                    return np.array([left_line, right_line])

            except:
                continue    
        

# 4. Main function
def P1_Main(video): 
    
    # 4.1. Read in video
    cap = cv2.VideoCapture(video)
    # Save Output
    fourcc = 0x7634706d
    out = cv2.VideoWriter('Output.mp4', fourcc, 25.0, (960, 540))

    # 4.2. Process by frame
    while (cap.isOpened):
        # 4.3. If the frame is read correctly
        ret, frame = cap.read()
        if ret == False:
            break

        # 4.4. Perform Canny edge detection
        canny_frame = Canny_Edge_Detection(frame)
        # 4.5. Mask the ROI
        roi = Region_of_Interest(canny_frame)
        # 4.6. Find and average the lines
        lines = cv2.HoughLinesP(roi, 2, np.pi/180, 100, np.array([]), minLineLength = 10, maxLineGap = 10)
        average_lines = Average_Params(frame, lines)
        # 4.7. Draw lines
        lined_frame = Display_Lines(frame, average_lines)
        overlayed_frame = cv2.addWeighted(frame, 0.8, lined_frame, 1, 1)

        # Save Output
        out.write(overlayed_frame)

        # 4.8. Result playback
        cv2.imshow('Result', overlayed_frame)
        # 4.9. Press space to exit
        if cv2.waitKey(40) & 0xFF == ord(' '):
            break
    # 4.10. Free buffer
    cap.release()
    out.release()
    cv2.destroyAllWindows()
        
    return "Done!"

In [7]:
P1_Main("solidYellowLeft.mp4")
# P1_Main("solidWhiteRight.mp4")
# P1_Main("challenge.mp4")

'Done!'